In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils import data
import time
from DNN import Dataset, train_epoch, eval_epoch, eval_epoch_true, FC, Seq2Seq, Seq2Seq_Attn 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Data for time

In [4]:
# training_data = torch.load("data_time/pems_bay/training_time.pt").float()
# test_data = torch.load("data_time/pems_bay/test_time.pt").float()
# training_data = torch.load("data_time/pems_sd/training_time.pt").float()
# test_data = torch.load("data_time/pems_sd/test_time.pt").float()

### Data for space 

In [2]:
# training_data = torch.load("data_space/training_space.pt").float()
# test_data = torch.load("data_space/test_space.pt").float()
training_data = torch.load("data_space/training_space_region.pt").float()
test_data = torch.load("data_space/test_space_region.pt").float()

In [3]:
training_data.shape, test_data.shape 

(torch.Size([159768, 3, 24, 20]), torch.Size([53256, 3, 24, 20]))

### Normalization 

In [4]:
mean = torch.tensor([training_data[:, 0, :, :].mean(), training_data[:, 1, :, :].mean(), training_data[:, 2, :, :].mean()])
std = torch.tensor([training_data[:, 0, :, :].std(), training_data[:, 1, :, :].std(), training_data[:, 2, :, :].std()])
training_norm = torch.zeros(training_data.shape, dtype = torch.float) 
training_norm[:, 0, :, :] = (training_data[:, 0, :, :] - mean[0]) / std[0]
training_norm[:, 1, :, :] = (training_data[:, 1, :, :] - mean[1]) / std[1]
training_norm[:, 2, :, :] = (training_data[:, 2, :, :] - mean[2]) / std[2] 

test_norm = torch.zeros(test_data.shape, dtype = torch.float) 
test_norm[:, 0, :, :] = (test_data[:, 0, :, :] - mean[0]) / std[0]
test_norm[:, 1, :, :] = (test_data[:, 1, :, :] - mean[1]) / std[1]
test_norm[:, 2, :, :] = (test_data[:, 2, :, :] - mean[2]) / std[2]

In [5]:
training_set = Dataset(training_norm)
test_set = Dataset(test_norm) 
training_set, val_set = data.random_split(training_set, [int(len(training_set) * 0.875), int(len(training_set) - int(len(training_set) * 0.875))])

In [6]:
# data loader 
train_loader = data.DataLoader(training_set, batch_size = 512, shuffle = True)
val_loader = data.DataLoader(val_set, batch_size = 512, shuffle = False) 
test_loader = data.DataLoader(test_set, batch_size = 512, shuffle = False) 

In [7]:
print(len(training_set))
print(len(val_set))
print(len(test_set))

139797
19971
53256


### Training 

In [20]:
# train model 

trial_num = 3 
preds_total = [] 
test_losses = [] 
num_epoch = 100 

for i in range(trial_num): 
    # build model 
    model = Seq2Seq_Attn(input_dim = 60, hidden_dim = 512, output_dim = 60, num_layers = 1, device = device).to(device) 
#     model = Seq2Seq(input_dim = 60, hidden_dim = 512, output_dim = 60, num_layers = 1).to(device) 
#     model = FC(input_dim = 60, input_len = 12, hidden_dim = 512, output_dim = 60).to(device) 
#     model = Latent_ODE(latent_dim = 256, obs_dim = 240, nhidden = 512, rhidden = 512, aug = False).to(device)
    name = "Seq2Seq_Attn"
    learning_rate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma=0.95)
    criterion = nn.MSELoss()
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    print("------Trial", i + 1)
    best_loss = 100   
    train_losses = []
    val_losses = [] 
    
    for epoch in range(1, num_epoch + 1): 
        start = time.time()
        train_loss = train_epoch(model, train_loader, optimizer, criterion)[-1]
        train_losses.append(train_loss)
        _, _, val_loss = eval_epoch(model, val_loader, criterion) 
        val_losses.append(val_loss)
        if val_loss <= best_loss: 
            best_loss = val_loss 
            best_model = model 
#             torch.save({"preds": preds, "trues": trues, "model": best_model}, "best_time_" + name + str(i+1) + ".pt") 
        end = time.time()
        preds, trues, test_loss = eval_epoch_true(best_model, test_loader, criterion, std, mean) 
        print("Epoch:", epoch, "completed in:", (end - start), "s. Training loss:", train_loss, ". Val loss:", val_loss, ". Test loss:", test_loss) 
        if (len(train_losses) > 50 and np.mean(val_losses[-5:]) >= np.mean(val_losses[-10:-5])):
            break
        scheduler.step() 
        if epoch % 5 == 0: print(optimizer.param_groups[0]['lr']) 
    
    # save the best model, prediction and ground truth 
    preds, trues, test_loss = eval_epoch_true(best_model, test_loader, criterion, std, mean) 
    torch.save({"preds": preds, "trues": trues, "model": best_model}, "result/Seq2Seq_Attn/best_space_region" + name + str(i) + ".pt") 
    preds_total.append(preds)
    test_losses.append(test_loss)
    print(test_loss)  

6938980
------Trial 1
Epoch: 1 completed in: 12.939138174057007 s. Training loss: 0.2728724391845456 . Val loss: 0.18476229533553123 . Test loss: 1.629836395411612
Epoch: 2 completed in: 12.683636665344238 s. Training loss: 0.13175742765956552 . Val loss: 0.12771276161074638 . Test loss: 1.376729307320319
Epoch: 3 completed in: 12.655012607574463 s. Training loss: 0.11363615745501797 . Val loss: 0.11912829447537661 . Test loss: 1.3345872110661796
Epoch: 4 completed in: 12.824986219406128 s. Training loss: 0.10534510984472985 . Val loss: 0.10404864363372326 . Test loss: 1.3467061493493138
Epoch: 5 completed in: 12.911186218261719 s. Training loss: 0.09797057887389712 . Val loss: 0.1052913449704647 . Test loss: 1.31489837302477
0.007737809374999999
Epoch: 6 completed in: 12.877658605575562 s. Training loss: 0.09283876254556388 . Val loss: 0.09621848333626985 . Test loss: 1.308874728654446
Epoch: 7 completed in: 12.92129921913147 s. Training loss: 0.09152788490763981 . Val loss: 0.0992036

Epoch: 56 completed in: 13.227051734924316 s. Training loss: 0.023009125483188318 . Val loss: 0.030743526248261332 . Test loss: 1.6136313103596776
Epoch: 57 completed in: 13.31382131576538 s. Training loss: 0.022829327453607624 . Val loss: 0.030301398038864134 . Test loss: 1.602203489688986
Epoch: 58 completed in: 13.284939527511597 s. Training loss: 0.022451513283716067 . Val loss: 0.030198209453374147 . Test loss: 1.600114384488949
Epoch: 59 completed in: 13.284379482269287 s. Training loss: 0.02220173318781992 . Val loss: 0.029903429187834264 . Test loss: 1.6072787374686275
Epoch: 60 completed in: 13.277870655059814 s. Training loss: 0.021993643757173397 . Val loss: 0.029667142778635025 . Test loss: 1.6166917211635223
0.00046069798986951934
Epoch: 61 completed in: 13.245464563369751 s. Training loss: 0.021702700444109685 . Val loss: 0.029588049137964846 . Test loss: 1.6046083308389265
Epoch: 62 completed in: 13.265088319778442 s. Training loss: 0.02158395049372946 . Val loss: 0.0296

Epoch: 11 completed in: 13.153262853622437 s. Training loss: 0.06734979434115608 . Val loss: 0.07347200382500887 . Test loss: 1.448272498880203
Epoch: 12 completed in: 13.15029764175415 s. Training loss: 0.06568099683436164 . Val loss: 0.06833633864298463 . Test loss: 1.3664702198093812
Epoch: 13 completed in: 13.16793155670166 s. Training loss: 0.06290486159931569 . Val loss: 0.06737783569842577 . Test loss: 1.463687484900175
Epoch: 14 completed in: 13.311776638031006 s. Training loss: 0.0597538969812602 . Val loss: 0.06551543166860938 . Test loss: 1.4008836554637503
Epoch: 15 completed in: 13.167620420455933 s. Training loss: 0.05833538865031552 . Val loss: 0.06416697949171066 . Test loss: 1.3310890019808237
0.00463291230159753
Epoch: 16 completed in: 13.10506010055542 s. Training loss: 0.05499936614430299 . Val loss: 0.05863631283864379 . Test loss: 1.4130540654183483
Epoch: 17 completed in: 13.14641809463501 s. Training loss: 0.051956002824824224 . Val loss: 0.05702039776369929 . T

Epoch: 66 completed in: 13.174572467803955 s. Training loss: 0.015528265233865402 . Val loss: 0.02283902750350535 . Test loss: 1.6162479391240474
Epoch: 67 completed in: 13.601306676864624 s. Training loss: 0.01540345648290032 . Val loss: 0.02285602637566626 . Test loss: 1.6188216496980585
Epoch: 68 completed in: 13.232865571975708 s. Training loss: 0.015306687465168698 . Val loss: 0.022775819431990384 . Test loss: 1.6214233196056758
Epoch: 69 completed in: 13.263936519622803 s. Training loss: 0.015168141872778425 . Val loss: 0.02266347426921129 . Test loss: 1.6210417319710113
Epoch: 70 completed in: 13.181136131286621 s. Training loss: 0.015048033676796803 . Val loss: 0.02262782109901309 . Test loss: 1.620790300884894
0.00027583690436774953
Epoch: 71 completed in: 13.221373081207275 s. Training loss: 0.01495220225617072 . Val loss: 0.02260178015567362 . Test loss: 1.6245585441051507
Epoch: 72 completed in: 13.153115034103394 s. Training loss: 0.014883165862955098 . Val loss: 0.0225625

Epoch: 21 completed in: 13.384044408798218 s. Training loss: 0.04509551139262906 . Val loss: 0.049910007044672966 . Test loss: 1.3892015553878234
Epoch: 22 completed in: 13.3307626247406 s. Training loss: 0.04279304478643802 . Val loss: 0.04752143518999219 . Test loss: 1.405409864821162
Epoch: 23 completed in: 13.343639135360718 s. Training loss: 0.04085792038236221 . Val loss: 0.0460353490896523 . Test loss: 1.417949080527841
Epoch: 24 completed in: 13.402570724487305 s. Training loss: 0.03941833497072658 . Val loss: 0.04370355186983943 . Test loss: 1.4133921570098118
Epoch: 25 completed in: 13.374975681304932 s. Training loss: 0.0374443986432722 . Val loss: 0.042001869156956674 . Test loss: 1.4238003830172814
0.0027738957312183374
Epoch: 26 completed in: 13.425501585006714 s. Training loss: 0.03639352136719836 . Val loss: 0.04195376038551331 . Test loss: 1.4282160929062124
Epoch: 27 completed in: 13.654013633728027 s. Training loss: 0.034812934166432295 . Val loss: 0.0404059507884085

Epoch: 76 completed in: 13.43778944015503 s. Training loss: 0.015138648616924991 . Val loss: 0.022343089431524278 . Test loss: 1.5169422590789632
Epoch: 77 completed in: 13.315095663070679 s. Training loss: 0.015098048353662892 . Val loss: 0.022288185730576515 . Test loss: 1.5132263263843209
Epoch: 78 completed in: 13.346868991851807 s. Training loss: 0.01502702112248453 . Val loss: 0.022292685508728028 . Test loss: 1.5129464038387963
Epoch: 79 completed in: 13.315927267074585 s. Training loss: 0.014957192754984772 . Val loss: 0.02222245940938592 . Test loss: 1.5141444130980601
Epoch: 80 completed in: 13.34012770652771 s. Training loss: 0.014893501303600568 . Val loss: 0.022251472948119043 . Test loss: 1.5162538340523102
0.00016515374385013573
Epoch: 81 completed in: 13.323540687561035 s. Training loss: 0.014828821715565712 . Val loss: 0.02220911714248359 . Test loss: 1.5143486214602886
Epoch: 82 completed in: 13.366432905197144 s. Training loss: 0.014772377976209578 . Val loss: 0.0221

In [14]:
preds, trues, test_loss = eval_epoch_true(best_model, test_loader, criterion, std, mean) 
torch.save({"preds": preds, "trues": trues, "model": best_model}, "result_bay/Seq2Seq/best_time_" + name + str(2) + ".pt") 

In [ ]:
a = torch.ones((1, 512, 1024))
b = torch.zeros((1, 512, 1024))
torch.cat((a, b), dim = 2).shape 

### Evaluation 

In [ ]:
rmse_losses = []
for preds_i in preds_total: 
#     print(preds_i.shape)
    rmse_losses.append(torch.sqrt(criterion(torch.tensor(preds_i), torch.tensor(trues))))

In [ ]:
rmse_losses

In [ ]:
np.array(rmse_losses).mean()

In [ ]:
# sensor location 

xi = [0,  45,  56,  75,  81,  86,  89,  95, 100, 105, 109, 112, 117,
       124, 128, 133, 137, 141, 146, 149, 152, 158, 163, 167, 171, 174,
       180, 186, 192, 197, 200, 205, 207, 210, 211, 213, 214, 228, 231,
       237, 240, 242, 251, 254, 258, 262, 266, 270, 277, 279, 282, 283,
       286, 288, 291, 294, 296, 298, 300, 303, 308, 310, 315, 317, 320,
       322, 327, 338, 342, 345, 352, 356, 359, 362, 366, 368, 374, 379] 

In [ ]:
## Plot the last time step from the first test sample 

preds_t = np.stack(preds_total)
preds_mean = preds_t.mean(axis = 0)
gt = trues[0]
loss = np.array(test_losses).mean()

In [ ]:
std_1 = preds_t.std(axis = 0)[-1, 0, -1]
std_2 = preds_t.std(axis = 0)[-1, 1, -1]
std_3 = preds_t.std(axis = 0)[-1, 2, -1]

In [ ]:
result = preds_mean[-1, :, -1, :]
std = [std_1, std_2, std_3]
true = trues[-1][:, -1, :]

In [ ]:
import matplotlib.pyplot as plt

tp=24
plt.style.use('default')
label={0: "Density $k$ (veh/m)",
       1: "Flow $q$ (veh/s)",
       2: "Speed $u$ (m/s)"} 

from matplotlib import gridspec

fwyp=[xi[n]* 300 / 1.e3 for n in range(len(xi))]

#gs = gridspec.GridSpec(3, 1, height_ratios=[2, 1, 1]) 
gs_kw={"height_ratios": [3, 3, 3]}

fig4,ax4=plt.subplots(figsize=(5, 15), nrows=3, gridspec_kw=gs_kw, sharex=True)
fig4.subplots_adjust(left=0.08, bottom=0.08, right=0.98, top=0.95)
ymin=[-0.01, -0.1, 20]
ymax=[0.06, 1.5, 40]


for n in range(3): 
    ax4[n].plot(fwyp, (true[n, :]), label='observed', marker='o', markersize=5)
    ax4[n].plot(fwyp, (result[n, :]), label='predicted', marker='o', markersize=5)
    ax4[n].fill_between(fwyp, (result[n, :] - std[n]), (result[n, :] + std[n]),  color='red', alpha=.3)    
    ax4[n].set_ylim(ymin[n], ymax[n])
    #ax4[n].set_xlabel("Distance (grid points)")
    ax4[n].set_ylabel(label[n])

    #ax4[n].set_xlim(fwyp[0]-1, fwyp[-1]+1)

#     lastpoint=-100
#     if n==0:
#         for p in range(len(xi)):
#             if (fwyp[p] - lastpoint) > 1.:
#                 ax4[n].text(fwyp[p], (y_pred[n, :, tp-68])[p] + 0.02, "none", rotation=90, fontsize=8)
#                 lastpoint=fwyp[p]
ax4[0].set_title("FC: last timestep from the last test sample")    
ax4[2].set_xlabel("Relative distance from North to South (km)")
#ax4[n].set_ylabel("Traffic density (Veh/m)")
fig4.savefig('FC.jpg', dpi=300)